<a href="https://colab.research.google.com/github/Volks44/VITMMA19/blob/main/Pretrain_RestNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras import applications
from keras.datasets import cifar10
import numpy as np
from matplotlib import pyplot

In [2]:
# load the pre-shuffled train and test data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

170498071/170498071 [==============================] - 5s 0us/step


In [3]:
# rescale [0,255] --> [0,1]
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [4]:
# one-hot encode the labels
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# break training set into training and validation sets
(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]

In [5]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )

# compute the data augmentation on the training set
datagen.fit(x_train)

In [12]:
base_model = applications.resnet50.ResNet50(weights= None, include_top=True, input_shape= (32, 32, 3), classes=10)

In [11]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 38, 38, 3)            0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 16, 16, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 16, 16, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [14]:
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

# training
batch_size = 64
epochs=100

checkpointer = ModelCheckpoint(filepath='model.RestNet50.hdf5', verbose=1, save_best_only=True)

optimizer = Adam(learning_rate=0.0005)

base_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = base_model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), callbacks=[checkpointer],
                steps_per_epoch=x_train.shape[0] // batch_size, epochs=epochs,verbose=2,
                validation_data=(x_test,y_test))

Epoch 1/100

Epoch 1: val_loss improved from inf to 1.70044, saving model to model.RestNet50.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


703/703 - 92s - loss: 1.9131 - accuracy: 0.3416 - val_loss: 1.7004 - val_accuracy: 0.3943 - 92s/epoch - 130ms/step
Epoch 2/100

Epoch 2: val_loss did not improve from 1.70044
703/703 - 48s - loss: 1.6542 - accuracy: 0.4271 - val_loss: 1.7478 - val_accuracy: 0.3944 - 48s/epoch - 68ms/step
Epoch 3/100

Epoch 3: val_loss did not improve from 1.70044
703/703 - 47s - loss: 1.5056 - accuracy: 0.4768 - val_loss: 2.1238 - val_accuracy: 0.3355 - 47s/epoch - 68ms/step
Epoch 4/100

Epoch 4: val_loss did not improve from 1.70044
703/703 - 47s - loss: 1.4814 - accuracy: 0.4946 - val_loss: 4.7801 - val_accuracy: 0.3908 - 47s/epoch - 67ms/step
Epoch 5/100

Epoch 5: val_loss did not improve from 1.70044
703/703 - 45s - loss: 1.5633 - accuracy: 0.4678 - val_loss: 11.9510 - val_accuracy: 0.2953 - 45s/epoch - 65ms/step
Epoch 6/100

Epoch 6: val_loss did not improve from 1.70044
703/703 - 45s - loss: 1.4571 - accuracy: 0.5028 - val_loss: 3.6496 - val_accuracy: 0.1673 - 45s/epoch - 64ms/step
Epoch 7/100

E